In [2]:
!pip install deepface
!pip numpy
!pip pandas
!pip opencv-python
!pip matplotlib
!pip tf-keras

ERROR: unknown command "numpy"
ERROR: unknown command "pandas"
ERROR: unknown command "opencv-python"
ERROR: unknown command "matplotlib"
ERROR: unknown command "tf-keras"


In [3]:
from deepface import DeepFace
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil

In [4]:
# List of available backends, models, and distance metrics
backends = ["opencv", "ssd", "dlib", "mtcnn", "retinaface"]
models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
metrics = ["cosine", "euclidean", "euclidean_l2"]

In [17]:
import zipfile
with zipfile.ZipFile('images.zip', 'r') as zip_ref:
  zip_ref.extractall('Datas')

In [11]:
# Path to the image for face recognition
img = "img/prafool.jpeg"
output_folder = "matched_images"
db_path = 'Datas/'

os.makedirs(output_folder, exist_ok=True)

def face_recognition(img):
    # Perform face recognition on the provided image
    # Find faces and identify people using a specific model and distance metric
    people = DeepFace.find(img_path=img, db_path=db_path, model_name=models[2], distance_metric=metrics[0])

    # Display the original image
    # plt.imshow(cv2.imread(img))

    # Print the identities of the recognized people
    for person in people:
      print(f'------------------ {person}')
      print(person['identity'][0].split('/')[1])


    for i, person_df in enumerate(people):
        print(f'-------- Matches for face {i + 1}: {len(person_df)} match(es) found --------')
        for index, row in person_df.iterrows():
            identity_path = row['identity']
            print(identity_path)


def face_recognition_v2(img):
    # Perform face recognition using DeepFace
    results = DeepFace.find(img_path=img, db_path=db_path, model_name=models[2], distance_metric=metrics[1])

    # Get base name of query image (e.g., "klaus.png")
    query_image_name = os.path.basename(img)

    # results is a list of DataFrames (one for each face found)
    for df in results:
        for _, row in df.iterrows():
            matched_path = row['identity']
            matched_filename = os.path.basename(matched_path)

            # Skip the query image itself
            if matched_filename == query_image_name:
                continue

            # Copy matched file to output folder
            destination_path = os.path.join(output_folder, matched_filename)
            shutil.copy(matched_path, destination_path)
            print(f"Matched and copied: {matched_filename}")

# Perform face recognition on a single image
face_recognition_v2(img)

25-06-02 06:53:44 - Found 14 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations: 100%|██████████| 14/14 [05:48<00:00, 24.91s/it]


25-06-02 06:59:33 - There are now 163 representations in ds_model_facenet512_detector_opencv_aligned_normalization_base_expand_0.pkl
25-06-02 06:59:33 - Searching img/prafool.jpeg in 163 length datastore
25-06-02 06:59:36 - find function duration 352.0928318500519 seconds
Matched and copied: IMG_0133.jpeg
Matched and copied: IMG_2108.jpeg
Matched and copied: IMG_6114.jpeg
Matched and copied: IMG_2054.jpeg
Matched and copied: smit_prince_uv.jpeg
Matched and copied: IMG_2108.jpeg
Matched and copied: IMG_3790.jpeg
Matched and copied: IMG_6079.jpeg
Matched and copied: IMG_6079.jpeg
Matched and copied: IMG_2108.jpeg
Matched and copied: IMG_6079.jpeg
Matched and copied: IMG_2054.jpeg
Matched and copied: IMG_3790.jpeg
Matched and copied: IMG_0147.jpeg
Matched and copied: IMG_2108.jpeg
Matched and copied: IMG_2108.jpeg
Matched and copied: IMG_5305.jpeg
Matched and copied: IMG_3790.jpeg
Matched and copied: IMG_2108.jpeg
Matched and copied: IMG_2108.jpeg
Matched and copied: IMG_2108.jpeg
Matched

In [ ]:
# | Model      | Metric        | Suggested Max Distance |
# | ---------- | ------------- | ---------------------- |
# | VGG-Face   | cosine        | 0.4 - 0.5              |
# | Facenet512 | euclidean\_l2 | 0.6 - 0.7              |
# | ArcFace    | cosine        | 0.3 - 0.4              |
# | SFace      | euclidean\_l2 | 0.7                    |

# Refer this above table, for threshold distance, according to the model used.

output_folder = "matched_images"
os.makedirs(output_folder, exist_ok=True)

models = ["VGG-Face", "Facenet", "Facenet512", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "SFace"]
metrics = ["cosine", "euclidean", "euclidean_l2"]

def face_recognition_v2(img):
    db_path = 'Datas/'
    model_name = models[2]          # Facenet512
    distance_metric = metrics[2]    # euclidean_l2
    max_threshold = 0.6             # Tune this (for Facenet512 + euclidean_l2)

    # Perform face recognition
    results = DeepFace.find(
        img_path=img,
        db_path=db_path,
        model_name=model_name,
        distance_metric=distance_metric,
        enforce_detection=False
    )

    query_image_name = os.path.basename(img)

    for df in results:
        for _, row in df.iterrows():
            if row["distance"] > max_threshold:
                continue  # Skip weak matches

            matched_path = row["identity"]
            matched_filename = os.path.basename(matched_path)

            if matched_filename == query_image_name:
                continue  # Skip matching itself

            destination_path = os.path.join(output_folder, matched_filename)
            shutil.copy(matched_path, destination_path)
            print(f"Matched and copied: {matched_filename} (Distance: {row['distance']:.4f})")

# Call the function
face_recognition_v2("img/prafool.jpeg")

25-06-02 09:24:51 - Found 20 newly added image(s), 0 removed image(s), 0 replaced image(s).


Finding representations:  45%|████▌     | 9/20 [06:37<10:08, 55.31s/it]

In [ ]:
# Prebuilt db cache

from deepface import DeepFace
import pickle

# Run once to cache DB embeddings
embeddings = DeepFace.represent(
    img_path="Data/",
    model_name="Facenet512",
    enforce_detection=False
)

# Save to disk
with open("db_embeddings.pkl", "wb") as f:
    pickle.dump(embeddings, f)


# Load embeddings and run search
with open("db_embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)

results = DeepFace.find(
    img_path=img,
    model_name="Facenet512",
    distance_metric="euclidean_l2",
    db_path="Data/",
    enforce_detection=False,
    stored_embeddings=embeddings,
    detector_backend="retinaface"
)


In [ ]:
# Use this for video streaming related stuff.

def realtime_face_recognition():
    # Define a video capture object
    vid = cv2.VideoCapture(0)

    while True:
        # Capture the video frame by frame
        ret, frame = vid.read()

        # Perform face recognition on the captured frame
        # Find faces and identify people using a specific model and distance metric
        people = DeepFace.find(img_path=frame, db_path=db_path, model_name=models[2], distance_metric=metrics[2], enforce_detection=False)

        for person in people:
            # Retrieve the coordinates of the face bounding box
            x = person['source_x'][0]
            y = person['source_y'][0]
            w = person['source_w'][0]
            h = person['source_h'][0]

            # Draw a rectangle around the face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Get the person's name and display it on the image
            name = person['identity'][0].split('/')[1]
            cv2.putText(frame, name, (x, y), cv2.FONT_ITALIC, 1, (0, 0, 255), 2)

        # Display the resulting frame
        cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('frame', 960, 720)
        cv2.imshow('frame', frame)

        # Check if the 'q' button is pressed to quit the program
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close all windows
    vid.release()
    cv2.destroyAllWindows()

# Perform real-time face recognition using the webcam
# realtime_face_recognition()